In [1]:
import pandas as pd
import numpy as np
import torch 
import sys
sys.path.append("../../")
from datasets import *
from models import *
from trainer import *

In [ ]:
claim_data = torch.Tensor(pd.read_pickle('../../data/claim_tensor.pkl'))
county_data = torch.Tensor(pd.read_pickle('../../data/county_tensor.pkl'))
hospitalizations_data = torch.Tensor(pd.read_pickle('../../data/hospitalizations.pkl'))
distance_matrix = torch.Tensor(pd.read_pickle('../../data/distance_mat.pkl'))
data_time = pd.read_pickle('../../data/date_range.pkl') #这个是list
claim_data.shape, county_data.shape, hospitalizations_data.shape, distance_matrix.shape,

In [ ]:
dynamic_data = torch.cat((claim_data,torch.unsqueeze(hospitalizations_data, -1)), -1)
static_data = county_data
label = torch.unsqueeze(hospitalizations_data, -1)
dynamic_data = dynamic_data[:500]
static_data = static_data[:500]
label = label[:500]

dynamic_data.shape, static_data.shape, label.shape

# New load way

In [ ]:
preprocess = PreprocessData(dynamic_data = dynamic_data, static_data = static_data, label = label, input_window = 7, output_window = 3, type= 'temporal', mode = 0, method = 'mean')
trainset, valset, testset = preprocess.getMlDataSetWithoutDivide()
train_temporal_loader = get_dataloader(trainset)
val_temporal_loader = get_dataloader(valset)
test_temporal_loader = get_dataloader(testset)


In [ ]:
arima_model = ARIMA()
arima_trainer = MlTrainer(arima_model)
arima_trainer.train(train_temporal_loader)
arima_trainer.evaluate(val_dataloader=val_temporal_loader)

In [ ]:
# y_train = torch.squeeze(train_label).to(torch.int)
y_train = torch.squeeze(preprocess.train_label)
y_train = torch.mean(y_train,0).to(torch.int)
print(y_train.shape)
N = y_train[0] * 10000
I0 = y_train[0]
R0 = I0
S0 = N - I0 - R0
# print(S0, I0, R0)

sir_model = SIR(S0=S0, I0= I0, R0=R0)
sir_trainer = MlTrainer(sir_model)
sir_trainer.train(train_dataloader = train_temporal_loader)
sir_trainer.evaluate(val_dataloader= val_temporal_loader)

In [ ]:
trainset, valset, testset = preprocess.getMlDataSet()
train_temporal_loader = get_dataloader(trainset)
val_temporal_loader = get_dataloader(valset)
test_temporal_loader = get_dataloader(testset)

In [ ]:
xgb = XGBoost()
xgb_trainer = MlTrainer(xgb)
xgb_trainer.train(train_dataloader=train_temporal_loader)
xgb_trainer.evaluate(val_dataloader=val_temporal_loader)

# Original load way

In [ ]:
num_positions = dynamic_data.shape[0]
spatio_indexes = []
for i in range(3):
    spatio_indexes.append(list(range(0,num_positions)))
train_dynamic_data, train_static_data, train_label,val_dynamic_data, val_static_data, val_label, test_dynamic_data, test_static_data, test_label = split_by_spatio_temporal(dynamic_data= dynamic_data, static_data=static_data,label=label,temporal_rate=[0.6,0.2,0.2], spatio_indexes=spatio_indexes)
train_dynamic_data.shape, train_static_data.shape, train_label.shape, val_dynamic_data.shape, val_static_data.shape, val_label.shape, test_dynamic_data.shape, test_static_data.shape, test_label.shape
train_tps = TemporalDataPreprocess(train_dynamic_data, train_static_data, train_label, mode = 0, method = 'sum')
val_tps = TemporalDataPreprocess(val_dynamic_data, val_static_data, val_label, mode = 0, method = 'sum')
test_tps = TemporalDataPreprocess(test_dynamic_data, test_static_data, test_label, mode = 0, method = 'sum')
train_data = train_tps.getMlData()
train_label = train_tps.getMlLabel()
val_data = val_tps.getMlData()
val_label = val_tps.getMlLabel()
train_data.shape, train_label.shape, val_data.shape, val_label.shape

In [ ]:
model = ARIMA()
model.train(train_data, train_label)
model.validate(val_data, val_label)

In [ ]:
y_train = torch.squeeze(train_label).to(torch.int)
N = y_train[0] * 10000
I0 = y_train[0]
R0 = I0
S0 = N - I0 - R0
print(S0, I0, R0)

y_val = torch.squeeze(val_label).to(torch.int)

model = SIR(S0=S0, I0= I0, R0=R0)
model.train(train_data, y_train)
model.validate(val_data, y_val)

In [ ]:
y_train = torch.squeeze(train_label).to(torch.int)
y_val = torch.squeeze(val_label).to(torch.int)
N = y_train[0] * 10000
I0 = y_train[0]
R0 = I0
E0 = I0 * 10
S0 = N - I0 - R0 - E0
print(S0, I0, R0)
model = SEIR(S0,E0,I0,R0)
model.train(train_data, y_train)
model.validate(val_data, y_val)

In [ ]:
model = XGBoost()
model.train(train_data, train_label)
model.validate(val_data, val_label)